# Klydo Scraper Debug Notebook

Use this to sanity-check the Klydo scraper locally (search + PDP).

Prereqs:
- Run inside this repo root.
- Ensure `.env` has `KLYDO_DEFAULT_SCRAPER=klydo` (optional but recommended) and a valid token if you have one (`KLYDO_KLYDO_API_TOKEN`).
- Kernel should point to the project venv (e.g., `.venv`).


In [5]:
import asyncio, json, os
from klydo.scrapers import get_scraper
from klydo.config import settings


In [6]:
# Quick visibility into the active env
print({
    'default_scraper': settings.default_scraper,
    'klydo_api_token_set': bool(settings.klydo_api_token),
    'klydo_session_id': settings.klydo_session_id,
    'env_file': settings.model_config.get('env_file'),
})


{'default_scraper': 'klydo', 'klydo_api_token_set': True, 'klydo_session_id': '1770335995922-654962', 'env_file': '.env'}


In [7]:
def pretty(label, payload):
    print(f"\n{label}:")
    print(json.dumps(payload, indent=2))

async def run_search(scraper, query: str, limit: int = 10):
    results = await scraper.search(query, limit=limit)
    print(f"Found {len(results)} results for '{query}'")
    for p in results:
        print(f"- {p.id} | {p.brand} | {p.name} | Rs {p.price.current}")
    return results

async def run_pdp(scraper, product_ids):
    for pid in product_ids:
        prod = await scraper.get_product(pid)
        if prod:
            pretty(pid, prod.model_dump(mode='json'))
        else:
            print(f"\n{pid}: None (not found)")


In [8]:
SCRAPER_NAME = 'klydo'
SEARCH_QUERY = 'Color Capital Athletic Jacket'
PRODUCT_IDS = [
    'STL_781SXKGTCJI6UVXG8HH0',
    'STL_NTPNGOCNMCCLKR5RJ9RZ',
    'STL_YW4USEN0V3O39HB1HYJ9',
]

async def main():
    scraper = get_scraper(SCRAPER_NAME)
    try:
        await run_search(scraper, SEARCH_QUERY, limit=10)
        await run_pdp(scraper, PRODUCT_IDS)
    finally:
        await scraper.close()

await main()


2026-02-22 16:50:13.658 | DEBUG    | klydo.scrapers.klydo_store:__init__:42 | KlydoStoreScraper initialized | session_id=1770335995922-654962
2026-02-22 16:50:13.659 | DEBUG    | klydo.logging:log_cache_hit:91 | Cache HIT: search:Color Capital Athletic Jacket:10
2026-02-22 16:50:13.661 | DEBUG    | klydo.logging:log_cache_hit:91 | Cache HIT: product:STL_781SXKGTCJI6UVXG8HH0
2026-02-22 16:50:13.662 | DEBUG    | klydo.logging:log_cache_miss:96 | Cache MISS: product:STL_NTPNGOCNMCCLKR5RJ9RZ
2026-02-22 16:50:13.662 | DEBUG    | klydo.logging:log_api_call:105 | API call: GET Klydo /catalog/pdp


Found 3 results for 'Color Capital Athletic Jacket'
- STL_YW4USEN0V3O39HB1HYJ9 | Color Capital | Navy Seamless Athletic Tennis Jacket | Rs 890
- STL_KILEOHMXRE0G1B4VU4GW | Color Capital | Brown Knitted Sporty Jacket with Front Zip | Rs 890
- STL_781SXKGTCJI6UVXG8HH0 | Color Capital | Greyish-Blue SkinnyFit Athletic Jacket | Rs 890

STL_781SXKGTCJI6UVXG8HH0:
{
  "id": "STL_781SXKGTCJI6UVXG8HH0",
  "name": "Greyish-Blue SkinnyFit Athletic Jacket",
  "brand": "Color Capital",
  "price": {
    "current": "890",
    "original": "990",
    "currency": "INR",
    "discount_percent": 10
  },
  "image_url": "https://storage.klydo.in/cdn-cgi/image/format=webp,width=400/product/stg0sy1hzkk1prgpsiem1taz.jpg",
  "category": "Fashion",
  "source": "Klydo",
  "url": "https://www.klydo.in/product/STL_781SXKGTCJI6UVXG8HH0",
  "description": "",
  "images": [
    {
      "url": "https://storage.klydo.in/cdn-cgi/image/format=webp,width=400/product/stg0sy1hzkk1prgpsiem1taz.jpg",
      "alt": "Greyish-Blue

2026-02-22 16:50:13.948 | DEBUG    | klydo.logging:log_api_call:105 | API call: GET Klydo /catalog/product
2026-02-22 16:50:14.023 | DEBUG    | klydo.logging:log_api_call:105 | API call: GET Klydo /catalog/products/STL_NTPNGOCNMCCLKR5RJ9RZ
2026-02-22 16:50:14.086 | DEBUG    | klydo.logging:log_api_call:105 | API call: GET Klydo /catalog/styles/STL_NTPNGOCNMCCLKR5RJ9RZ
2026-02-22 16:50:14.249 | DEBUG    | klydo.scrapers.klydo_store:get_product:243 | Attempting search fallback for STL_NTPNGOCNMCCLKR5RJ9RZ
2026-02-22 16:50:14.252 | DEBUG    | klydo.logging:log_cache_hit:91 | Cache HIT: search:STL_NTPNGOCNMCCLKR5RJ9RZ:10
2026-02-22 16:50:14.269 | DEBUG    | klydo.logging:log_cache_hit:91 | Cache HIT: product:STL_YW4USEN0V3O39HB1HYJ9
2026-02-22 16:50:14.269 | DEBUG    | klydo.scrapers.klydo_store:close:479 | Closing KlydoStoreScraper



STL_NTPNGOCNMCCLKR5RJ9RZ: None (not found)

STL_YW4USEN0V3O39HB1HYJ9:
{
  "id": "STL_YW4USEN0V3O39HB1HYJ9",
  "name": "Navy Seamless Athletic Tennis Jacket",
  "brand": "Color Capital",
  "price": {
    "current": "890",
    "original": "990",
    "currency": "INR",
    "discount_percent": 10
  },
  "image_url": "https://storage.klydo.in/cdn-cgi/image/format=webp,width=400/product/stghjuf82dtetevlj02sqm86.jpg",
  "category": "Fashion",
  "source": "Klydo",
  "url": "https://www.klydo.in/product/STL_YW4USEN0V3O39HB1HYJ9",
  "description": "",
  "images": [
    {
      "url": "https://storage.klydo.in/cdn-cgi/image/format=webp,width=400/product/stghjuf82dtetevlj02sqm86.jpg",
      "alt": "Navy Seamless Athletic Tennis Jacket"
    }
  ],
  "sizes": [],
  "colors": [],
  "rating": null,
  "review_count": 0,
  "in_stock": true,
  "specifications": {}
}


In [9]:
# Optional: swap in other queries/IDs here
ALT_QUERY = 'T-Shirts'
ALT_IDS = ['STL_KLOIFTGI59HN8UC9YVJ8']

async def alt_run():
    scraper = get_scraper(SCRAPER_NAME)
    try:
        await run_search(scraper, ALT_QUERY, limit=5)
        await run_pdp(scraper, ALT_IDS)
    finally:
        await scraper.close()

# Uncomment to run
await alt_run()

2026-02-22 16:50:29.927 | DEBUG    | klydo.scrapers.klydo_store:__init__:42 | KlydoStoreScraper initialized | session_id=1770335995922-654962
2026-02-22 16:50:29.929 | DEBUG    | klydo.logging:log_cache_miss:96 | Cache MISS: search:T-Shirts:5
2026-02-22 16:50:29.930 | DEBUG    | klydo.logging:log_api_call:105 | API call: GET Klydo /catalog/search
2026-02-22 16:50:30.368 | DEBUG    | klydo.scrapers.klydo_store:search:161 | Klydo search complete | query=T-Shirts | results=5
2026-02-22 16:50:30.369 | DEBUG    | klydo.logging:log_cache_miss:96 | Cache MISS: product:STL_KLOIFTGI59HN8UC9YVJ8
2026-02-22 16:50:30.369 | DEBUG    | klydo.logging:log_api_call:105 | API call: GET Klydo /catalog/pdp
2026-02-22 16:50:30.496 | DEBUG    | klydo.logging:log_api_call:105 | API call: GET Klydo /catalog/product


Found 5 results for 'T-Shirts'
- STL_EUK3NFAK5YPZ03AZVF1U | Styli | Men Front Chest Embroidered Oversize Fit Terry T-Shirt | Rs 599
- STL_5IMHWKB2HR46XO5AD2JP | Styli | Men Front Printed Oversize Fit Terry T-Shirt | Rs 599
- STL_M1G3866GJX7DACS73Y86 | Styli | Men Front Printed Oversize Fit Terry T-Shirt | Rs 599
- STL_I33Q9WU20RR54UKZBGRZ | Styli | Men Front Text Printed Oversize Fit Terry T-Shirt | Rs 599
- STL_VVX0IGV4RE060XJ6FTLN | Outzidr | White Cropped Boxy T-Shirt With Graphic | Rs 449


2026-02-22 16:50:30.596 | DEBUG    | klydo.logging:log_api_call:105 | API call: GET Klydo /catalog/products/STL_KLOIFTGI59HN8UC9YVJ8
2026-02-22 16:50:30.672 | DEBUG    | klydo.logging:log_api_call:105 | API call: GET Klydo /catalog/styles/STL_KLOIFTGI59HN8UC9YVJ8
2026-02-22 16:50:30.762 | DEBUG    | klydo.scrapers.klydo_store:get_product:243 | Attempting search fallback for STL_KLOIFTGI59HN8UC9YVJ8
2026-02-22 16:50:30.764 | DEBUG    | klydo.logging:log_cache_miss:96 | Cache MISS: search:STL_KLOIFTGI59HN8UC9YVJ8:10
2026-02-22 16:50:30.764 | DEBUG    | klydo.logging:log_api_call:105 | API call: GET Klydo /catalog/search
2026-02-22 16:50:32.348 | DEBUG    | klydo.scrapers.klydo_store:search:161 | Klydo search complete | query=STL_KLOIFTGI59HN8UC9YVJ8 | results=10
2026-02-22 16:50:32.349 | DEBUG    | klydo.scrapers.klydo_store:close:479 | Closing KlydoStoreScraper



STL_KLOIFTGI59HN8UC9YVJ8: None (not found)
